<a href="https://colab.research.google.com/github/syedanida/NeuralNetworks_with_Keras/blob/main/2_Advanced_Keras_Deep_Learning_Constructs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

def custom_lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = keras.callbacks.LearningRateScheduler(custom_lr_schedule)
print("Custom Learning Rate Scheduler created. Use it in model.fit() as follows:")
print("model.fit(..., callbacks=[lr_scheduler])")


Custom Learning Rate Scheduler created. Use it in model.fit() as follows:
model.fit(..., callbacks=[lr_scheduler])


In [ ]:
# Notebook: Part 2ii – Custom Dropout (MCAlphaDropout) Example

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class MCAlphaDropout(layers.Layer):
    def __init__(self, rate, **kwargs):
        super(MCAlphaDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

# Example usage within a model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(784,)),
    MCAlphaDropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │          50,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_alpha_dropout (MCAlphaDropout)    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Notebook: Part 2iii – Custom Normalization (MaxNormDense) Example

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class MaxNormDense(layers.Layer):
    def __init__(self, units, max_value=2, **kwargs):
        super(MaxNormDense, self).__init__(**kwargs)
        self.units = units
        self.max_value = max_value

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        w_norm = tf.clip_by_norm(self.w, self.max_value)
        return tf.matmul(inputs, w_norm) + self.b

# Example usage in a model
model = keras.Sequential([
    MaxNormDense(64, max_value=3, input_shape=(784,)),
    layers.Activation('relu'),
    MaxNormDense(10)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


<ipython-input-3-b23d89903195>:9: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(MaxNormDense, self).__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ max_norm_dense (MaxNormDense)        │ (None, 64)                  │          50,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_norm_dense_1 (MaxNormDense)      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Notebook: Part 2iv – TensorBoard Integration Example

import tensorflow as tf
from tensorflow import keras

log_dir = "logs/advanced/"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print("TensorBoard callback created. Use it with model.fit() like so:")
print("model.fit(..., callbacks=[tensorboard_callback])")


TensorBoard callback created. Use it with model.fit() like so:
model.fit(..., callbacks=[tensorboard_callback])


In [ ]:
# Notebook: Part 2v – Custom Loss Function (CustomHuberLoss) Example

import tensorflow as tf
from tensorflow import keras

class CustomHuberLoss(keras.losses.Loss):
    def __init__(self, delta=1.0, **kwargs):
        super(CustomHuberLoss, self).__init__(**kwargs)
        self.delta = delta

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        abs_error = tf.abs(error)
        quadratic = tf.minimum(abs_error, self.delta)
        linear = abs_error - quadratic
        loss = 0.5 * tf.square(quadratic) + self.delta * linear
        return tf.reduce_mean(loss)

# Example usage:
loss_instance = CustomHuberLoss(delta=1.0)
print("CustomHuberLoss created. Use in model.compile(loss=loss_instance, ...)")


CustomHuberLoss created. Use in model.compile(loss=loss_instance, ...)


In [ ]:
# Notebook: Part 2vi – Custom Activation, Initializer, Regularizer & Constraint Example

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Custom activation (variant of Leaky ReLU)
def my_leaky_relu(x, alpha=0.2):
    return tf.maximum(alpha * x, x)

# Custom initializer (variant of Glorot initializer)
def my_glorot_initializer(shape, dtype=None):
    fan_in, fan_out = shape[0], shape[1]
    scale = np.sqrt(2.0 / (fan_in + fan_out))
    return tf.random.uniform(shape, minval=-scale, maxval=scale, dtype=dtype)

# Custom L1 regularizer (wrapped as a Keras regularizer)
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, l1=1e-4):
        self.l1 = l1

    def __call__(self, x):
        return self.l1 * tf.reduce_sum(tf.abs(x))

    def get_config(self):
        return {'l1': float(self.l1)}


# Custom weight constraint that enforces positive weights
class PositiveWeightConstraint(keras.constraints.Constraint):
    def __call__(self, w):
        return tf.clip_by_value(w, 0.0, tf.reduce_max(w))

# Example layer using custom functions
custom_layer = layers.Dense(
    64,
    activation=lambda x: my_leaky_relu(x, alpha=0.2),
    kernel_initializer=my_glorot_initializer,
    kernel_regularizer=MyL1Regularizer(l1=1e-4), # Use the custom regularizer class
    kernel_constraint=PositiveWeightConstraint()
)

model = keras.Sequential([
    custom_layer,
    layers.Dense(10, activation='softmax')
])
model.build(input_shape=(None, 784))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 64)                  │          50,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Notebook: Part 2vii – Custom Metric (HuberMetric) Example

import tensorflow as tf
from tensorflow import keras

class HuberMetric(keras.metrics.Metric):
    def __init__(self, delta=1.0, name="huber_metric", **kwargs):
        super(HuberMetric, self).__init__(name=name, **kwargs)
        self.delta = delta
        self.sum_metric = self.add_weight(name="sum", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = y_true - y_pred
        abs_error = tf.abs(error)
        quadratic = tf.minimum(abs_error, self.delta)
        linear = abs_error - quadratic
        loss = 0.5 * tf.square(quadratic) + self.delta * linear
        self.sum_metric.assign_add(tf.reduce_sum(loss))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.sum_metric / self.count

    def reset_states(self):
        self.sum_metric.assign(0.0)
        self.count.assign(0.0)

print("Custom Huber Metric defined. Use it with model.compile(metrics=[HuberMetric(delta=1.0)])")


Custom Huber Metric defined. Use it with model.compile(metrics=[HuberMetric(delta=1.0)])


In [ ]:
# Notebook: Part 2viii – Custom Layers Example

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Custom Exponential Layer
class ExponentialLayer(layers.Layer):
    def call(self, inputs):
        return tf.exp(inputs)

# Custom Dense Layer
class MyDense(layers.Layer):
    def __init__(self, units):
        super(MyDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Custom layer that adds Gaussian Noise
class AddGaussianNoise(layers.Layer):
    def __init__(self, stddev):
        super(AddGaussianNoise, self).__init__()
        self.stddev = stddev

    def call(self, inputs, training=False):
        if training:
            noise = tf.random.normal(tf.shape(inputs), mean=0.0, stddev=self.stddev)
            return inputs + noise
        return inputs

# Use built-in LayerNormalization
layer_norm = layers.LayerNormalization()

# Build a model combining custom layers
model = keras.Sequential([
    MyDense(64),
    AddGaussianNoise(0.1),
    ExponentialLayer(),
    layer_norm,
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ my_dense (MyDense)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ add_gaussian_noise                   │ ?                           │     0 (unbuilt) │
│ (AddGaussianNoise)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ exponential_layer (ExponentialLayer) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class ResidualBlock(layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.dense1 = layers.Dense(units, activation='relu')
        self.dense2 = layers.Dense(units)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return layers.add([inputs, x])

class ResidualRegressor(keras.Model):
    def __init__(self, units):
        super(ResidualRegressor, self).__init__()
        self.block1 = ResidualBlock(units)
        self.block2 = ResidualBlock(units)
        self.out = layers.Dense(1)

    def call(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        return self.out(x)

# For this approach, dummy input should have 64 features:
residual_model = ResidualRegressor(64)
dummy_input = tf.random.normal((1, 64))  # now shape is (1,64)
_ = residual_model(dummy_input)
residual_model.summary()


Model: "residual_regressor_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ residual_block_6 (ResidualBlock)     │ ?                           │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_block_7 (ResidualBlock)     │ ?                           │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (1, 1)                      │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,705 (65.25 KB)

 Trainable params: 16,705 (65.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Notebook: Part 2x – Custom Optimizer (MyMomentumOptimizer) Example

import tensorflow as tf
from tensorflow import keras

class MyMomentumOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, momentum=0.9, name="MyMomentumOptimizer", **kwargs):
        super(MyMomentumOptimizer, self).__init__(name, **kwargs)
        self.learning_rate = learning_rate
        self.momentum = momentum

    def apply_gradients(self, grads_and_vars, name=None, experimental_aggregate_gradients=True):
        for grad, var in grads_and_vars:
            if grad is not None:
                var.assign_sub(self.learning_rate * grad)

    def get_config(self):
        return {"learning_rate": self.learning_rate, "momentum": self.momentum}

print("Custom Momentum Optimizer defined. Use it in model.compile(), e.g.:")
print("model.compile(optimizer=MyMomentumOptimizer(learning_rate=0.01, momentum=0.9), ...)")


Custom Momentum Optimizer defined. Use it in model.compile(), e.g.:
model.compile(optimizer=MyMomentumOptimizer(learning_rate=0.01, momentum=0.9), ...)


In [ ]:
# Notebook: Part 2xi – Custom Training Loop for Fashion MNIST Example

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Preprocess the data: flatten and normalize
x_train = x_train.reshape(-1, 28 * 28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype('float32') / 255.0

# Build a simple model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(10, activation='softmax')
])

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 100 == 0:
            print(f"Step {step}: Loss = {loss_value.numpy():.4f}")

print("Custom training loop complete.")


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
Step 0: Loss = 2.4929
Step 100: Loss = 0.6536
Step 200: Loss = 0.4238
Step 300: Loss = 0.5853
Step 400: Loss = 0.5259
Step 500: Loss = 0.4992
Step 600: Loss = 0.4347
Step 700: Loss = 0.6206
Step 800: Loss = 0.5358
Step 900: Loss = 0.4961
Epoch 2/3
Step 0: Loss = 0.3074
Step 100: Loss = 0.3770
Step 200: Loss = 0.3345
Step 300: Loss = 0.4483
Step 400: Loss = 0.4304
Step 500: Loss = 0.3724
Step 600: Loss = 0.3775
Step 700: Loss = 0.5551
Step 800: Loss = 0.4697
Step 900: Loss = 0.4867
Epoch 3/3
Step 0: Loss = 0.2615
Step 100: Loss = 0.3395
Step 200: Loss = 0.2770
Step 300: Loss = 0.3766
Step 400: Loss = 0.4015
Step 500: Loss = 0.3332
Step 600: Loss = 0.3260
Step 700: Loss = 0.5320
Step 800: Loss = 0.4073
Step 900: Loss = 0.4439
Custom training loop complete.
